In [1]:

import os
import zipfile
import shutil
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
local_zip = '/content/drive/My Drive/Thesis/KDEF_and_AKDEF.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
try:
    os.makedirs('/content/Thesis_Emodet', exist_ok=True)
except OSError as error:
    print(error)
zip_ref.extractall('/content/Thesis_Emodet')
zip_ref.close()

In [0]:
files = []
dir = []
basepath = '/content/Thesis_Emodet/KDEF_and_AKDEF/KDEF'
for entry in os.listdir(basepath):
    sub=[]
    dir.append(basepath+'/'+entry+'/')
    for file in os.listdir(basepath+'/'+entry):
        sub.append(file)
    files.append(sub)

root='/content/Thesis_Emodet/emotions/'
emotion_type=['AF','AN','DI','HA','NE','SA','SU']

for sublist in range(len(files)):
    for file in files[sublist]:
        for i in range (len(emotion_type)):
            if file[4:6] == emotion_type[i]:
                try:
                    os.makedirs(root+emotion_type[i], exist_ok=True)
                except OSError as error:
                    print(error)
                shutil.copy(dir[sublist]+file, root+emotion_type[i])
                break

In [0]:
basepath2 = '/content/Thesis_Emodet/KDEF_and_AKDEF/AKDEF'
files_AKDEF = []
for entry in os.listdir(basepath2):
    files_AKDEF.append(entry)
for file in files_AKDEF:
    for i in range (len(emotion_type)):
        if file[1:3] == emotion_type[i]:
            try:
                os.makedirs(root+emotion_type[i], exist_ok=True)
            except OSError as error:
                print(error)
                #print(root+emotion_type[i]+'/'+file)
            shutil.copy(basepath2+'/'+file, root+emotion_type[i])
            break

In [0]:
# Call_back function for stopping training after a certain level

class myCallback(tf.keras.callbacks.Callback):# Your Code
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

AccuracyLimiterCallback = myCallback()

In [0]:
checkpoint_path = "/content/drive/My Drive/Thesis/test_2 cp/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For data split into test & validation
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.15) # set validation split

batch_size=32
img_height=572
img_width=422
train_data_dir='/content/Thesis_Emodet/emotions/' # source directory for training images
train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    #color_mode='grayscale',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # training data

validation_generator = train_datagen.flow_from_directory(
    directory=train_data_dir, # same directory as training data
    #color_mode='grayscale',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # validation data




Found 4225 images belonging to 7 classes.
Found 742 images belonging to 7 classes.


In [8]:

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (572 , 422, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed8')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2020-02-18 06:45:32--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.128, 2607:f8b0:400c:c13::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/content/Thesis_Emo 100%[===================>]  83.84M   204MB/s    in 0.4s    

2020-02-18 06:45:32 (204 MB/s) - ‘/content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
last layer output shape:  (None, 16, 11, 1280)


In [9]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.4)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (7, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 572, 422, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 285, 210, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 285, 210, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 285, 210, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            epochs = 500,
            callbacks=[cp_callback,AccuracyLimiterCallback],
            verbose = 1)
history.save ('/content/drive/My Drive/Thesis/test_2 cp/test_2.h5')

Epoch 1/500
133/133 [==============================] - 251s 2s/step - loss: 4.0369 - acc: 0.1536 - val_loss: 1.9450 - val_acc: 0.1577
Epoch 2/500
133/133 [==============================] - 230s 2s/step - loss: 1.9619 - acc: 0.1484 - val_loss: 1.9464 - val_acc: 0.1334
Epoch 3/500
133/133 [==============================] - 230s 2s/step - loss: 1.9537 - acc: 0.1453 - val_loss: 1.9455 - val_acc: 0.1442
Epoch 4/500
133/133 [==============================] - 230s 2s/step - loss: 1.9540 - acc: 0.1439 - val_loss: 1.9452 - val_acc: 0.1482
Epoch 5/500
133/133 [==============================] - 230s 2s/step - loss: 1.9516 - acc: 0.1482 - val_loss: 1.9460 - val_acc: 0.1456
Epoch 6/500
133/133 [==============================] - 237s 2s/step - loss: 1.9498 - acc: 0.1472 - val_loss: 1.9455 - val_acc: 0.1536
Epoch 7/500
133/133 [==============================] - 251s 2s/step - loss: 1.9450 - acc: 0.1543 - val_loss: 1.9459 - val_acc: 0.1550
Epoch 8/500
133/133 [==============================] - 258s 2s

In [1]:
model.save ('/content/drive/My Drive/Thesis/test_2 cp/test_2.h5')

NameError: ignored

In [0]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()